In [1]:
import torch
import shap
import pandas as pd
from setfit import SetFitModel
from transformers import AutoTokenizer

from lib.utils import load_jsonl_file


def get_device():
  """Returns the appropriate device available in the system: CUDA, MPS, or CPU"""
  if torch.backends.mps.is_available():
    return torch.device("mps")
  elif torch.cuda.is_available():
    return torch.device("cuda")
  else:
    return torch.device("cpu")


# Get best device
device = get_device()

CLASS_NAMES = ['support', 'oppose']  # 0/1

model_id = "sentence-transformers/paraphrase-mpnet-base-v2"  # Example model_id

# Load dataset
DATASET = load_jsonl_file("shared_data/dataset_2_test.jsonl")

support_class = [data for data in DATASET if data["label"] == "support"]
oppose_class = [data for data in DATASET if data["label"] == "oppose"]

# Load model
model_setfit_path = "models/22"
model = SetFitModel.from_pretrained(model_setfit_path, local_files_only=True, device=device)

# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)


def f(x):  # x is a list of strings
  # Make predictions using the model
  predictions = model.predict_proba(x)
  return predictions.cpu()


explainer = shap.Explainer(
  model=f,  # prediction function
  masker=tokenizer, 
  output_names=["support", "oppose"],
  algorithm="auto",
  linearize_link=None,
  seed=42
)

texts = [data["text"] for data in oppose_class]

sentence = texts[5:6]
print(sentence)

shap_values = explainer(sentence, fixed_context=None)
# print(shap_values)

"""
# Make predictions using the model and get the predicted class name
predictions = model.predict_proba(sentence)
predicted_class_index = np.argmax(predictions, axis=1)[0]
predicted_class_name = CLASS_NAMES[predicted_class_index]
print(f"Predicted class: {predicted_class_name}")"""

shap.plots.text(shap_values)

summed_shap_values = shap_values.mean(0).values
summed_shap_features = shap_values.mean(0).feature_names

# flattened_shap_values
summed_shap_values = [row[0] for row in summed_shap_values]

# Convert to DataFrame for easy CSV saving
df_shap_values = pd.DataFrame(summed_shap_values, columns=["Value"])

# Add a column for the feature names if available
df_shap_values["Feature"] = summed_shap_features
df_shap_values = df_shap_values.sort_values(by="Value", ascending=False)

print(df_shap_values)

# shap.plots.bar(shap_values)

Loading data from shared_data/dataset_2_test.jsonl...
Loaded 200 items.
['We made sure NATO was prepared to deter any aggression against any member state.']
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/272 [00:00<?, ?it/s]

           Value     Feature
13  1.054369e-01       state
6   7.511248e-02    prepared
12  5.949440e-02      member
1   1.616058e-02          We
7   1.173116e-02          to
4   8.899268e-03        NATO
2   8.653643e-03        made
3   6.140513e-03        sure
0  -2.536431e-08            
5  -8.315086e-03         was
9  -9.199328e-02         any
14 -1.493016e-01           .
10 -1.528225e-01  aggression
11 -1.813225e-01     against
8  -2.658122e-01       deter
